# Access Control

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Access%2520Control.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This document explains how to use Access Control in Matter, and will be updated as development proceeds.

## What Does and Doesn’t Work Right Now?

Briefly, you can read and write the entire ACL attribute in the all-clusters-app, but Access Control isn’t yet turned on, so it won’t affect interactions. There’s almost no error checking when writing the ACL attribute (e.g. ensuring subjects match auth mode, only your fabric can be written, etc.) so exercise caution for now.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [1]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.
os.system('rm -rf /tmp/chip_*')

0

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [2]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

 Replacing  store path ./credentials/development/paa-root-certs with 
/home/root/repos/connectedhomeip/credentials/development/paa-root-certs
Note that you are still running from 
/home/root/repos/connectedhomeip/docs/development_controllers/chip-repl so other relative
paths may be off.

[1737913722.154926][1138777:1138777] CHIP:CTL: Setting attestation nonce to random value
[1737913722.155127][1138777:1138777] CHIP:CTL: Setting CSR nonce to random value
[1737913722.167269][1138777:1138777] CHIP:DL: NVS set: chip-factory/unique-id = "10043DCE74BA9070"
[1737913722.168091][1138777:1138777] CHIP:DL: NVS set: chip-factory/vendor-id = 65521 (0xFFF1)
[1737913722.169011][1138777:1138777] CHIP:DL: NVS set: chip-factory/product-id = 32769 (0x8001)
[1737913722.169910][1138777:1138777] CHIP:DL: NVS set: chip-counters/reboot-count = 1 (0x1)
[1737913722.170825][1138777:1138777] CHIP:DL: NVS set: chip-counters/total-operational-hours = 0 (0x0)
[1737913722.171768][1138777:1138777] CHIP:DL: NVS set: chip-counters/boot-reason = 0 (0x0)
[1737913722.172574][1138777:1138777] CHIP:DL: NVS set: chip-config/regulatory-location = 0 (0x0)
[1737913722.173348][1138777:1138777] CHIP:DL: NVS set: chip-config/location-capability = 2 (0x2)
[1737913722.173536][1138777:1138777] CHIP:DL: Got Ethernet i

─────────────────────────────────────────────────── Matter REPL ───────────────────────────────────────────────────

            Welcome to the Matter Python REPL!

            For help, please type matterhelp()

            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.

            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-01-26 18:48:42 ThinkPad chip.storage[1138777] ERROR [Errno 2] No such file or directory: '/tmp/repl-storage.json'
2025-01-26 18:48:42 ThinkPad chip.storage[1138777] CRITICAL Could not load configuration from /tmp/repl-storage.json - resetting configuration...
2025-01-26 18:48:42 ThinkPad chip.storage[1138777] WARNING No valid SDK configuration present - clearing out configuration
2025-01-26 18:48:42 ThinkPad chip.storage[1138777] WARNING No valid REPL configuration present - clearing out configuration


The following objects have been created:

certificateAuthorityManager:    Manages a list of CertificateAuthority instances.
        caList:                         The list of CertificateAuthority instances.
        caList:                 A specific FabricAdmin object at index m for the nth CertificateAuthority instance.

Default CHIP Device Controller (NodeId: 112233): has been initialized to manage caList[0].adminList[0] (FabricId = 
1), and is available as devCtrl

## Commission and Setup Server

### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [3]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

# The location of the all-clusters-app in the cloud playground is one level higher - adjust for this by testing for file presence.
if (os.path.isfile('../../../out/linux-x64-all-clusters/chip-all-clusters-app')):
    appPath = '../../../out/linux-x64-all-clusters/chip-all-clusters-app'
else:
    appPath = '../../../../out/linux-x64-all-clusters/chip-all-clusters-app'

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

### Commission Target

Commission the target with a NodeId of 2.

In [4]:
await devCtrl.CommissionOnNetwork(2, 20202021)

2

## Bootstrap ACLs

During Commissioning, an ACL that assigns Administer rights to the commissioner(i.e. chip-repl) was automatically installed on the commissionee.

In [5]:
import chip.clusters as Clusters

await devCtrl.ReadAttribute(2, [ (0, Clusters.OperationalCredentials)], returnClusterObject=True)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: OperationalCredentials(
│   │   │   (data version)=2078601825,
│   │   │   NOCs=[
│   │   │   │   NOCStruct(
│   │   │   │   │   noc=b'\x150\x01\x01\x01$\x02\x017\x03$\x13\x02\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x15\x01$\x11\x02\x18$\x07\x01$\x08\x010\tA\x04#\xd0\xac\x80\x1d\xf3G\x1e\x9d\xf4\xb8v\x07"\x8eK\x0f}\x9b\xc7c\x96"\xf8{\xbb\x90\x8f\xa9\xe0\xacE=r\x86\xb8\xf7l\x84G\xbfm\xd5\x9f\xba\x8b\xd4\x14\x94(o\x00Jig8\xedh\xf7\x08\x82\xa8r{7\n5\x01(\x01\x18$\x02\x016\x03\x04\x02\x04\x01\x180\x04\x14\xd9\x04\x1f\xf5\xad@\x7f\x0fv\x8c\xe6t\xe8\n+`\xc5\x02n\xbd0\x05\x14\xfb\xb2\',L%E%6\x8eH\t?p\xe6=\x19\x16\xe6`\x180\x0b@s\x02U\xe3\x1f9\x11n\xc6\x1c,\x7f\xd8\x8e\xc6\x03\x04r\x15\x86\x9e\x0c\x14\xdde\xba`\xfd\xb9"\x10\xd0&\r/\xa4\x98\x87\xdb\xa6\xd5>\x95R\x14,\'\xea\'\xd85\xff\x03\'\xd6\xab\xec\x13m\x86<\x80\xa7\xe1\x18',
│   │   │   │   │   icac=b'\x150\x01\x01\x01$\x02\x017\x03$\x14\x01\x18&\x04\x80"\x81\'&\x05\x

### Automatically Installed ACL

In [6]:
data = await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl) ] )
data


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 91529008,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [7]:
acl = data[0][chip.clusters.Objects.AccessControl][chip.clusters.Objects.AccessControl.Attributes.Acl]
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   )
]

### Installing a CASE ACL

In [8]:
acl.append(Clusters.AccessControl.Structs.AccessControlEntryStruct(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kOperate,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kCase,
    targets = [ Clusters.AccessControl.Structs.AccessControlTargetStruct(
        endpoint = 1,
    ) ] ) )
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ],
│   │   fabricIndex=1
│   )
]

In [9]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )


[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [10]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 91529011,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   AccessControlTargetStruct(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   dev

### Installing a Group ACL

In [11]:
acl.append( Clusters.AccessControl.Structs.AccessControlEntryStruct(
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kManage,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kGroup,
    subjects = [ 123, 456 ],
    targets = [
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        cluster = Clusters.OnOff.id,
      ),
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        endpoint = 1,
      ),
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        cluster = Clusters.LevelControl.id,
        endpoint = 2,
      ) ] ) )
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ],
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kManage: 4>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kGroup: 3>,
│   │   subjects=[
│   │   │   123,
│   │   │   456
│   │   ],
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=6,
│   │   │   │   endpoint=Null,
│   │   │   │   deviceType=

In [12]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] ) 


[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [13]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 91529015,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   AccessControlTargetStruct(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   dev